In [72]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [73]:
def time_elapsed(sec):
    h = int(sec/3600)
    m = int(sec/60)
    s = sec % 60
    return "{}:{>02}:{:>05.2f}".format(h,m,s)

In [74]:
# CONFIGURATION DATA #

# Data generated must be square
IM_SIZE = 160 #
IM_CHANNELS = 3 #RGB
IM_SHAPE = (IM_SIZE, IM_SIZE, IM_CHANNELS)
FOLDER_PATH = '.'

# SIZE OF VECTOR TO GENERATE IMAGES FROM
SEED_SIZE = 140

# Preview image 
PREVIEW_ROWS = 4 #numbers of displayed images ROWS*COLS
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

EPOCHS = 1
BATCH_SIZE = 32
BUFFER_SIZE = 60000

print(f'Generated faces will be {IM_SIZE} px size')


Generated faces will be 160 px size


In [75]:
# loading training data
NPY_FILE_PATH = os.path.join(FOLDER_PATH, f'training_data_{IM_SIZE}_{IM_SIZE}.npy')

if not os.path.isfile(NPY_FILE_PATH): #if there is not file with data
    IMAGES_PATH = os.path.join(FOLDER_PATH,'images')
    training_data = []
    
    for filename in tqdm(os.listdir(IMAGES_PATH)): #load images
        path = os.path.join(IMAGES_PATH,filename)
        image = Image.open(path).resize((IM_SIZE,IM_SIZE), Image.ANTIALIAS)    
        training_data.append(np.asarray(image))
    
    training_data = np.reshape(training_data,(-1,IM_SIZE,IM_SIZE,IM_CHANNELS))
    training_data = training_data.astype(np.float32)
    training_data = training_data/127.5 - 1
    np.save(NPY_FILE_PATH, training_data) #save constructed data to save time next time
    
else: # if there is already created data, just load it
    training_data = np.load(NPY_FILE_PATH)
    

In [76]:
# Change datatype to tensorflow, so its easy to shuffle and slice into baths
train_dataset = tf.data.Dataset.from_tensor_slices(training_data
                ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [77]:
# buidling generator of images
def build_generator(seed_size, channels):
    model = Sequential()
    
    #watch size of data in every layer, end layer size must give 160x160x3
    model.add(Dense(5*5*256,activation = 'relu',input_dim=seed_size))
    model.add(Reshape((5,5,256))) # size is 5x5x5256
    
    model.add(UpSampling2D()) # size is 10x10x256
    model.add(Dropout(0.25))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D()) # size is 20x20x256
    model.add(Dropout(0.25))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D()) # size is 40x40x256
    model.add(Dropout(0.25))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D()) # size is 80x80x256
    model.add(Dropout(0.25))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D()) # size is 160x160x256
    model.add(Dropout(0.25))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same")) #160x160x3
    model.add(Activation("tanh"))
    
    return model

def build_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, 
                     padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dropout(0.50))
    model.add(Conv2D(1024, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

    

In [93]:
# build an objects of generator and discriminator
generator = build_generator(SEED_SIZE, IM_CHANNELS)

discriminator = build_discriminator(IM_SHAPE)


generator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)

In [79]:
# computing of loss funtion
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = fake_loss + real_loss
    return total_loss

def generator_loss(fake_output):
    total_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return total_loss

In [94]:
@tf.function
def train_step(images,epoch_nr):
    seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])
    

    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # output of generator
        generated_images = generator(seed, training = True)
        # output of discriminator
        real_output = discriminator(images, training = True)
        fake_output = discriminator(generated_images, training = True)

        # getting loss function
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output,fake_output)
        
        # compute gradients
        gradients_of_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        
        # back propagation
        generator_optimizer.apply_gradients(zip(gradients_of_gen,generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_disc,discriminator.trainable_variables))
        
        return gen_loss, disc_loss

In [95]:
def save_images(cnt,noise):
    image_array = np.full(( 
    PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
    PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
    255, dtype=np.uint8)
    generated_images = generator.predict(noise)

    generated_images = 0.5 * generated_images + 0.5

    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
            c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
            image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
            image_count += 1

          
    output_path = os.path.join(FOLDER_PATH,'output')
    if not os.path.exists(output_path):
        os.makedirs(output_path)
  
    filename = os.path.join(output_path,f"train-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [96]:
def train(dataset, epochs):
    start_time=time.time()
    fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))
    for epoch in range(epochs):
        epoch_s_time = time.time()
        gen_loss_list = []
        disc_loss_list = []
        
       
        for image_batch in dataset:
            t = train_step(image_batch,epoch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])
            
        g_loss = sum(gen_loss_list)/ len(gen_loss_list)
        d_loss = sum(gen_loss_list)/ len(gen_loss_list)
        epoch_e_time = time.time()
        epoch_time = time_elapsed(epoch_e_time - epoch_s_time)
        print(f'Epoch: {epoch}, gen_loss: {g_loss}, disc_loss: {d_loss}, TIME {epoch_time}')
        save_images(epoch,fixed_seed)

In [97]:
train(train_dataset, EPOCHS)

ResourceExhaustedError:  OOM when allocating tensor with shape[32,160,160,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_11/up_sampling2d_24/resize/ResizeNearestNeighbor (defined at C:\Users\Adrian\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_step_31827]

Function call stack:
train_step


(160, 160, 3)

NameError: name 'real_output' is not defined